run with jupyter notebook --NotebookApp.iopub_data_rate_limit=10000000000  playground.ipynb
download textblob: pip install -U textblob
python -m textblob.download_corpora

In [1]:
%reload_ext sql
%sql postgres://isdb@localhost/postgres

'Connected: isdb@postgres'

In [22]:
%%sql

SELECT * FROM TweetsRaw LIMIT 5;

5 rows affected.


tweet_id,author_id,inbound,created_at,tweet,response_tweet_id,in_response_to_tweet_id
1,sprintcare,False,2017-10-31 22:10:47,@115712 I understand. I would like to assist you. We would need to get you into a private secured link to further assist.,2,3
2,115712,True,2017-10-31 22:11:45,@sprintcare and how do you propose we do that,None,1
3,115712,True,2017-10-31 22:08:27,@sprintcare I have sent several private messages and no one is responding as usual,1,4
4,sprintcare,False,2017-10-31 21:54:49,@115712 Please send us a Private Message so that we can further assist you. Just click âMessageâ at the top of your profile.,3,5
5,115712,True,2017-10-31 21:49:35,@sprintcare I did.,4,6


In [25]:
%%sql

DROP VIEW IF EXISTS OriginalTweets;

CREATE VIEW OriginalTweets AS
SELECT * FROM TweetsRaw
WHERE in_response_to_tweet_id IS NULL;

Done.
Done.


[]

In [43]:
%%sql

DROP VIEW IF EXISTS TweetsReplied;

CREATE VIEW TweetsReplied AS
SELECT * FROM TweetsRaw
WHERE response_tweet_id IS NOT NULL;

Done.
Done.


[]

In [2]:
%%sql

DROP VIEW IF EXISTS TweetsNotReplied;

CREATE VIEW TweetsNotReplied AS
SELECT * FROM TweetsRaw
WHERE response_tweet_id IS NULL
LIMIT 5;

Done.
Done.


[]

In [29]:
%%sql

SELECT r.tweet_id, r.tweet, COUNT(o.*) AS replies
FROM OriginalTweets AS o
JOIN TweetsRaw AS r
ON r.in_response_to_tweet_id = o.tweet_id
GROUP BY r.tweet_id, r.tweet
ORDER BY replies DESC
LIMIT 5;

5 rows affected.


tweet_id,tweet,replies
1000001,@AppleSupport Is this allowed when you upgrade at an Apple store with the upgrade program?,1
1000003,@AppleSupport https://t.co/OnHabGwPo7,1
1000010,@AppleSupport Does this mean I donât have to back my old phone up?,1
1000011,@356751 Thanks for reaching out to us for support. We would love to help you. Please DM us so that we can look into this together. https://t.co/GDrqU22YpT,1
10,@115712 Hello! We never like our customers to feel like they are not valued.,1


In [36]:
%%sql

SELECT COUNT(*) FROM TweetsRaw;

1 rows affected.


count
2811774


In [44]:
%%sql
SELECT COUNT(*) FROM TweetsReplied;

1 rows affected.


count
1771145


In [42]:
%%sql
SELECT COUNT(*) FROM TweetsNotReplied;

1 rows affected.


count
1040629


In [53]:
from IPython.display import HTML
import base64  
import pandas as pd
from textblob import TextBlob

temp = %sql SELECT * FROM TweetsReplied;
replied = temp.DataFrame()

temp = %sql SELECT * FROM TweetsNotReplied;
notReplied = temp.DataFrame()

def numPeriods(row):
    ct = 0
    for i in range(len(row['tweet'])):
        if row['tweet'][i] == '.': ct += 1
    return ct

def numCommas(row):
    ct = 0
    for i in range(len(row['tweet'])):
        if row['tweet'][i] == ',': ct += 1
    return ct

def numExclamations(row):
    ct = 0
    for i in range(len(row['tweet'])):
        if row['tweet'][i] == '!': ct += 1
    return ct

def numQuestions(row):
    ct = 0
    for i in range(len(row['tweet'])):
        if row['tweet'][i] == '?': ct += 1
    return ct

def findPolarity(row):
    blob = TextBlob(row['tweet'])
    return blob.sentiment.polarity

def findSubjectivity(row):
    blob = TextBlob(row['tweet'])
    return blob.sentiment.subjectivity

5 rows affected.
5 rows affected.


In [54]:
replied['Periods'] = replied.apply (lambda row: numPeriods(row),axis=1)
replied['Commas'] = replied.apply (lambda row: numCommas(row),axis=1)
replied['Exclamations'] = replied.apply (lambda row: numExclamations(row),axis=1)
replied['Questions'] = replied.apply (lambda row: numQuestions(row),axis=1)
replied['Polarity'] = replied.apply (lambda row: findPolarity(row),axis=1)
replied['Subjectivity'] = replied.apply (lambda row: findSubjectivity(row),axis=1)

notReplied['Periods'] = notReplied.apply (lambda row: numPeriods(row),axis=1)
notReplied['Commas'] = notReplied.apply (lambda row: numCommas(row),axis=1)
notReplied['Exclamations'] = notReplied.apply (lambda row: numExclamations(row),axis=1)
notReplied['Questions'] = notReplied.apply (lambda row: numQuestions(row),axis=1)
notReplied['Polarity'] = notReplied.apply (lambda row: findPolarity(row),axis=1)
notReplied['Subjectivity'] = notReplied.apply (lambda row: findSubjectivity(row),axis=1)

In [55]:
def create_download_link( df, title = "Download CSV file", filename = "NotReplied.csv"):  
    csv = notReplied.to_csv(index =False)
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()
    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
    html = html.format(payload=payload,title=title,filename=filename)
    return HTML(html)

create_download_link(notReplied)

In [56]:
from IPython.display import HTML
import base64  
import pandas as pd  

def create_download_link( df, title = "Download CSV file", filename = "Replied.csv"):  
    csv = replied.to_csv(index =False)
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()
    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
    html = html.format(payload=payload,title=title,filename=filename)
    return HTML(html)

create_download_link(replied)

In [58]:
replied

,tweet_id,author_id,inbound,created_at,tweet,response_tweet_id,in_response_to_tweet_id,Periods,Commas,Exclamations,Questions,Polarity,Subjectivity
0,1,sprintcare,False,2017-10-31 22:10:47,@115712 I understand. I would like to assist y...,2,3,3,0,0,0,0.000000,0.437500
1,3,115712,True,2017-10-31 22:08:27,@sprintcare I have sent several private messag...,1,4,0,0,0,0,-0.083333,0.208333
2,4,sprintcare,False,2017-10-31 21:54:49,@115712 Please send us a Private Message so th...,3,5,2,0,0,0,0.166667,0.458333
3,5,115712,True,2017-10-31 21:49:35,@sprintcare I did.,4,6,1,0,0,0,0.000000,0.000000
4,6,sprintcare,False,2017-10-31 21:46:24,@115712 Can you please send us a private messa...,"5,7",8,0,1,0,1,0.000000,0.437500
